In [2]:
import numpy as np
import polars as pl

gen_row = 5000
rng = np.random.default_rng(seed=7)

building_data={
    "sqft": rng.exponential(scale=1000, size = gen_row),
    "year": rng.integers(low=1975, high=2025, size=gen_row),
    "building_type": rng.choice(['A','B','C'], size=gen_row)
}

buildings = pl.DataFrame(building_data)
buildings

sqft,year,building_type
f64,i64,str
707.529256,1977,"""C"""
1025.203348,2020,"""C"""
568.548657,2006,"""A"""
895.109864,1984,"""A"""
206.532754,2004,"""A"""
…,…,…
710.435755,1990,"""C"""
408.872783,2001,"""C"""
57.562059,2018,"""C"""


In [4]:
buildings.schema

Schema([('sqft', Float64), ('year', Int64), ('building_type', String)])

In [5]:
buildings.head

<bound method DataFrame.head of shape: (5_000, 3)
┌─────────────┬──────┬───────────────┐
│ sqft        ┆ year ┆ building_type │
│ ---         ┆ ---  ┆ ---           │
│ f64         ┆ i64  ┆ str           │
╞═════════════╪══════╪═══════════════╡
│ 707.529256  ┆ 1977 ┆ C             │
│ 1025.203348 ┆ 2020 ┆ C             │
│ 568.548657  ┆ 2006 ┆ A             │
│ 895.109864  ┆ 1984 ┆ A             │
│ 206.532754  ┆ 2004 ┆ A             │
│ …           ┆ …    ┆ …             │
│ 710.435755  ┆ 1990 ┆ C             │
│ 408.872783  ┆ 2001 ┆ C             │
│ 57.562059   ┆ 2018 ┆ C             │
│ 3728.088949 ┆ 2020 ┆ C             │
│ 686.678345  ┆ 2005 ┆ C             │
└─────────────┴──────┴───────────────┘>

In [6]:
buildings.describe()

statistic,sqft,year,building_type
str,f64,f64,str
"""count""",5000.0,5000.0,"""5000"""
"""null_count""",0.0,0.0,"""0"""
"""mean""",994.094456,1999.5488,null
"""std""",1016.641569,14.39669,null
"""min""",1.133256,1975.0,"""A"""
"""25%""",286.807549,1987.0,null
"""50%""",669.406964,2000.0,null
"""75%""",1342.909782,2012.0,null
"""max""",9307.793917,2024.0,"""C"""


In [7]:
buildings.select("sqft")

sqft
f64
707.529256
1025.203348
568.548657
895.109864
206.532754
…
710.435755
408.872783
57.562059


In [9]:
buildings.select(pl.col("sqft").sort() / 1000)

sqft
f64
0.001133
0.001152
0.001429
0.001439
0.001505
…
6.931917
7.247539
7.629569


In [14]:
after_2000 = buildings.filter(pl.col("year")>2000)
after_2000.shape

(2457, 3)

In [16]:
after_2000.select(pl.col('year').min())

year
i64
2001


In [21]:
# In this example, you first call buildings.groupby("building_type"), 
# which creates a Polars GroupBy object. 
# The GroupBy object has an aggregation method, .agg(), 
# which accepts a list of expressions that are computed for each group. 
# For instance, pl.mean("sqft") calculates the average square footage for each building type, 
# and pl.count() returns the number of buildings of each building type. 
# You use .alias() to name the aggregated columns.

# While it’s not apparent with the high-level Python API, 
# all Polars expressions are optimized and run in parallel under the hood. 
# This means that Polars expressions don’t always run in the order you specify, 
# and they don’t necessarily run on a single core. 
# Instead, Polars optimizes the order in which expressions are evaluated in a query,
# and the work is spread across available cores. 
# You’ll see examples of optimized queries later in this tutorial.

In [23]:
buildings.group_by("building_type").agg(
    [
        pl.mean("sqft").alias("mean_sqft"),
        pl.median("year").alias("median_year"),
        pl.len()
    ]
)

building_type,mean_sqft,median_year,len
str,f64,f64,u32
"""C""",999.854722,2000.0,1692
"""B""",992.754444,2000.0,1655
"""A""",989.539918,2001.0,1653


In [52]:
import numpy as np
import polars as pl

gen_row = 5000
rng = np.random.default_rng(seed = 7)
building_data2 = {
    "sqft":rng.exponential(scale= 1000, size = gen_row),
    "price": rng.exponential(scale=100_000, size = gen_row),
    "year": rng.integers(low=1975,high=2030,size= gen_row),
    "type": rng.choice(['A','B','C','D','E'], size = gen_row)
}
bd = pl.LazyFrame(building_data2)
bd

In [53]:
lazyQuery = (
    bd.with_columns(
        (pl.col('price') / pl.col('sqft')).alias('price_per_sqft')
                       )
    .filter(
        (pl.col('price_per_sqft')>100) & (pl.col('year')<2000)
                       )
)

                           
lazyQuery

In [59]:
print(lazyQuery.explain())

FILTER [(col("price_per_sqft")) > (100.0)] FROM
   WITH_COLUMNS:
   [[(col("price")) / (col("sqft"))].alias("price_per_sqft")] 
    DF ["sqft", "price", "year", "type"]; PROJECT */4 COLUMNS; SELECTION: [(col("year")) < (2000)]


In [62]:
 (
     lazyQuery
     .collect()
     .select(pl.col(["price_per_sqft", "year"]))
     .describe()
 )

statistic,price_per_sqft,year
str,f64,f64
"""count""",1126.0,1126.0
"""null_count""",0.0,0.0
"""mean""",1222.929776,1987.082593
"""std""",5024.841734,7.270599
"""min""",100.072032,1975.0
"""25%""",166.103508,1981.0
"""50%""",294.024197,1987.0
"""75%""",715.72476,1994.0
"""max""",90314.966163,1999.0


In [63]:
import requests
import pathlib

def download_file(file_url: str, local_file_path: pathlib.Path) -> None:
    """Download a file and save it with the specified file name."""
    response = requests.get(file_url)
    if response:
        local_file_path.write_bytes(response.content)
        print(f"File successfully downloaded and stored at: {local_file_path}")
    else:
        raise requests.exceptions.RequestException(
            f"Failed to download the file. Status code: {response.status_code}"
        )

In [64]:
url = "https://data.wa.gov/api/views/f6w7-q2d2/rows.csv?accessType=DOWNLOAD"
local_file_path = pathlib.Path("electric_cars.csv")

download_file(url, local_file_path)

File successfully downloaded and stored at: electric_cars.csv


In [66]:
lazy_car_data = pl.scan_csv(local_file_path)
lazy_car_data.schema

C:\Users\Admin\AppData\Local\Temp\ipykernel_4632\3716273341.py:2: PerformanceWarning: Resolving the schema of a LazyFrame is a potentially expensive operation. Use `LazyFrame.collect_schema()` to get the schema without this warning.
  lazy_car_data.schema


Schema([('VIN (1-10)', String),
        ('County', String),
        ('City', String),
        ('State', String),
        ('Postal Code', Int64),
        ('Model Year', Int64),
        ('Make', String),
        ('Model', String),
        ('Electric Vehicle Type', String),
        ('Clean Alternative Fuel Vehicle (CAFV) Eligibility', String),
        ('Electric Range', Int64),
        ('Base MSRP', Int64),
        ('Legislative District', Int64),
        ('DOL Vehicle ID', Int64),
        ('Vehicle Location', String),
        ('Electric Utility', String),
        ('2020 Census Tract', Int64)])

In [ ]:
>>> lazy_car_query = (
...     lazy_car_data
...     .filter((pl.col("Model Year") >= 2018))
...     .filter(
...         pl.col("Electric Vehicle Type") == "Battery Electric Vehicle (BEV)"
...     )
...     .groupby(["State", "Make"])
...     .agg(
...         pl.mean("Electric Range").alias("Average Electric Range"),
...         pl.min("Model Year").alias("Oldest Model Year"),
...         pl.count().alias("Number of Cars"),
...     )
...     .filter(pl.col("Average Electric Range") > 0)
...     .filter(pl.col("Number of Cars") > 5)
...     .sort(pl.col("Number of Cars"), descending=True)
... )

>>> lazy_car_query.collect()
shape: (20, 5)
┌───────┬───────────┬────────────────────────┬───────────────────┬────────────────┐
│ State ┆ Make      ┆ Average Electric Range ┆ Oldest Model Year ┆ Number of Cars │
│ ---   ┆ ---       ┆ ---                    ┆ ---               ┆ ---            │
│ str   ┆ str       ┆ f64                    ┆ i64               ┆ u32            │
╞═══════╪═══════════╪════════════════════════╪═══════════════════╪════════════════╡
│ WA    ┆ TESLA     ┆ 89.114509              ┆ 2018              ┆ 55690          │
│ WA    ┆ NISSAN    ┆ 93.115056              ┆ 2018              ┆ 5267           │
│ WA    ┆ CHEVROLET ┆ 111.746651             ┆ 2018              ┆ 5001           │
│ WA    ┆ KIA       ┆ 65.380428              ┆ 2018              ┆ 3178           │
│ …     ┆ …         ┆ …                      ┆ …                 ┆ …              │
│ VA    ┆ TESLA     ┆ 139.133333             ┆ 2018              ┆ 15             │
│ MD    ┆ TESLA     ┆ 50.6                   ┆ 2018              ┆ 10             │
│ TX    ┆ TESLA     ┆ 94.625                 ┆ 2018              ┆ 8              │
│ NC    ┆ TESLA     ┆ 61.428571              ┆ 2018              ┆ 7              │
└───────┴───────────┴────────────────────────┴───────────────────┴────────────────┘

In [67]:
lazy_car_data.collect()

VIN (1-10),County,City,State,Postal Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,2020 Census Tract
str,str,str,str,i64,i64,str,str,str,str,i64,i64,i64,i64,str,str,i64
"""5YJ3E1EB0J""","""Thurston""","""Olympia""","""WA""",98512,2018,"""TESLA""","""MODEL 3""","""Battery Electric Vehicle (BEV)""","""Clean Alternative Fuel Vehicle…",215,0,35,104823078,"""POINT (-122.957046 46.991391)""","""PUGET SOUND ENERGY INC""",53067012730
"""WA1AAAGE9M""","""Kitsap""","""Port Orchard""","""WA""",98367,2021,"""AUDI""","""E-TRON""","""Battery Electric Vehicle (BEV)""","""Clean Alternative Fuel Vehicle…",222,0,35,156660507,"""POINT (-122.6530052 47.4739066…","""PUGET SOUND ENERGY INC""",53035092901
"""5YJ3E1EA2J""","""Yakima""","""Yakima""","""WA""",98902,2018,"""TESLA""","""MODEL 3""","""Battery Electric Vehicle (BEV)""","""Clean Alternative Fuel Vehicle…",215,0,14,269374108,"""POINT (-120.530331 46.59534)""","""PACIFICORP""",53077000500
"""5YJ3E1EA4N""","""Yakima""","""Yakima""","""WA""",98902,2022,"""TESLA""","""MODEL 3""","""Battery Electric Vehicle (BEV)""","""Eligibility unknown as battery…",0,0,15,213383894,"""POINT (-120.530331 46.59534)""","""PACIFICORP""",53077001202
"""7SAYGAEE2P""","""Snohomish""","""Bothell""","""WA""",98012,2023,"""TESLA""","""MODEL Y""","""Battery Electric Vehicle (BEV)""","""Eligibility unknown as battery…",0,0,1,229496046,"""POINT (-122.206146 47.839957)""","""PUGET SOUND ENERGY INC""",53061052009
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""JTMAB3FVXR""","""Snohomish""","""Snohomish""","""WA""",98290,2024,"""TOYOTA""","""RAV4 PRIME""","""Plug-in Hybrid Electric Vehicl…","""Clean Alternative Fuel Vehicle…",42,0,44,262809249,"""POINT (-122.0483457 47.9435765…","""PUGET SOUND ENERGY INC""",53061052402
"""7FCTGAAA7P""","""Pierce""","""Orting""","""WA""",98360,2023,"""RIVIAN""","""R1T""","""Battery Electric Vehicle (BEV)""","""Eligibility unknown as battery…",0,0,2,252195450,"""POINT (-122.197791 47.0948565)""","""PUGET SOUND ENERGY INC||CITY O…",53053070100
"""1V2GNPE87P""","""Spokane""","""Spokane""","""WA""",99201,2023,"""VOLKSWAGEN""","""ID.4""","""Battery Electric Vehicle (BEV)""","""Eligibility unknown as battery…",0,0,3,227314790,"""POINT (-117.428902 47.658268)""","""MODERN ELECTRIC WATER COMPANY""",53063002300
